In [2]:
import pandas as pd
import re
import string

In [3]:
scraped= pd.read_csv('Hotel Location Table Mapping.csv',encoding='latin1')
hotels=pd.read_csv('Hotel Location.csv',encoding='latin1')
scraped_ngrams=pd.read_csv('submission.csv',encoding='latin1')

old=pd.read_csv('old.csv',encoding='latin1')
pivot=pd.read_csv('pivot.csv',encoding='latin1')

In [5]:
scraped_ngrams.head(20)

,scraped_url,hotel_mapped,corpus,derived
0,https://www.flightcentre.co.za/product/14197346,NaN,adaaran select meedhupparu nan your maldives h...,"['adaaran select', 'select meedhupparu', 'meed..."
1,https://www.flightcentre.co.za/product/5415249,NaN,family fun on the wild coast nan your eastern ...,"['family fun', 'fun wild', 'wild coast', 'coas..."
2,https://www.flightcentre.co.za/product/16385011,NaN,azuri residences by life in blue nan your maur...,"['azuri residences', 'residences life', 'life ..."
3,https://www.flightcentre.co.za/product/16323810,NaN,seas the day in mauritius nan your mauritius h...,"['seas day', 'day mauritius', 'mauritius nan',..."
4,https://www.flightcentre.co.za/product/13250621,NaN,kruger park splendour nan your kruger national...,"['kruger park', 'park splendour', 'splendour n..."
5,https://www.flightcentre.co.za/product/15650253,NaN,hungarian f1 grand prix 2022 nan your hungary ...,"['hungarian f1', 'f1 grand', 'grand prix', 'pr..."
6,https://www.flightcentre.co.za/product/12615686,NaN,seas the day in mauritius nan your mauritius h...,"['seas day', 'day mauritius', 'mauritius nan',..."
7,https://www.flightcentre.co.za/product/12616094,NaN,clear skies turquoise sea nan your mauritius h...,"['clear skies', 'skies turquoise', 'turquoise ..."
8,https://www.flightcentre.co.za/product/15650322,NaN,wimbledon 2022 nan your england holiday packag...,"['wimbledon 2022', '2022 nan', 'nan england', ..."
9,https://www.flightcentre.co.za/product/15968357,NaN,tropical palms sandy shores nan your zanzibar ...,"['tropical palms', 'palms sandy', 'sandy shore..."


In [34]:
scraped.head()

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped
0,https://www.flightcentre.co.za/product/14197346,flightcentre,maldives,Adaaran Select Meedhupparu,\nYour Maldives Holiday Package includes:\nRet...,\nYour Maldives Holiday Package includes:\n\nR...,NaN,NaN
1,https://www.flightcentre.co.za/product/5415249,flightcentre,south africa,Family Fun on the Wild Coast,\nYour Eastern Cape Holiday Package includes:\...,\nYour Eastern Cape Holiday Package includes:\...,NaN,NaN
2,https://www.flightcentre.co.za/product/16385011,flightcentre,mauritius,Azuri Residences by Life in Blue,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN
3,https://www.flightcentre.co.za/product/16323810,flightcentre,mauritius,Seas the Day in Mauritius,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN
4,https://www.flightcentre.co.za/product/13250621,flightcentre,south africa,Kruger Park Splendour,\nYour Kruger National Park Holiday Package in...,\nYour Kruger National Park Holiday Package in...,NaN,NaN


In [35]:
hotels.head()

,Name,Address1,Address2,City,StateProvince,PostalCode,Country,Latitude,Longitude,AirportCode,PropertyCurrency,StarRating,Location
0,41,41 Buckingham Palace Road,NaN,London,NaN,SW1W 0PS,GB,51.49830,-0.14372,LHR,GBP,5.0,Near Buckingham Palace
1,#2 Andrews  Port Douglas,#2 Andrews Close,NaN,Port Douglas,QL,4877,AU,-16.51689,145.46770,CNS,AUD,5.0,Near Four Mile Beach Park
2,#2 Hastings Street,2 Hastings Street,NaN,Noosa Heads,QL,4567,AU,-26.38616,153.08722,MCY,AUD,4.0,In Noosa Heads (Noosa)
3,.Com,Albena,NaN,Albena,NaN,9620,BG,43.37024,28.07389,VAR,EUR,3.0,In Albena
4,?The Dunstane,4 West Coates & 5 Hampton Terrace,Haymarket,Edinburgh,NaN,EH12 5JQ,GB,55.94571,-3.22880,EDI,GBP,4.0,Near Murrayfield Stadium


In [3]:
def clean_text(text):
    """
    This function uses regular expressions to 
        - remove links characters
        - remove html characters,
        - remove escape sequences,
        - remove capitalization,
        - any extra white space from each text and then converts them to lowercase.

    Input:
    text: original text
          datatype: string

    Output:
    texts: modified text
           datatype: string
    """
    # replace links with url-web
    pattern_url = 'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = 'url-web'
    text = re.sub(pattern_url, subs_url, text)
    # replace the html characters with " "
    text = re.sub('<.*?>#()&', ' ', text)
    text = re.sub('&', ' ', text)
#     text = re.sub('(', ' ', text)
#     text = re.sub(')', ' ', text)
    
    text=re.sub('Nan', ' ', text)
    # replace escape sequence with space
    escape_seq = ["\r", "\n",'\a', '\b', '\f', "\`", '\”', '\t']
    for i in escape_seq:
        text = re.sub(i, " ", text)
    # Remove Capitalization
    text = text.lower()
    # plit and join the words
    text=' '.join(text.split())
    
    return text

In [5]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("#",' ',str(text))
    text = re.sub("@",'',str(text))
    
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]#", "", str(text))
    
    return text

In [6]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])


In [7]:
scraped['description'] = scraped['description'].astype(str).apply(clean_text)
scraped['inclusions'] = scraped['inclusions'].astype(str).apply(clean_text)
scraped['hotel_name'] = scraped['hotel_name'].astype(str).apply(clean_text)
scraped['title'] = scraped['title'].astype(str).apply(clean_text)

In [8]:
scraped['corpus'] = ''
corpus = []
columns = ['title','hotel_name','description', 'inclusions']
for i in range(0, len(scraped['scraped_url'])):
    words = ''
    for col in columns:
        words = words + scraped.iloc[i][col] + " "        
    corpus.append(words)
scraped['corpus'] = corpus
#scrapy.set_index('scraped_url', inplace=True)
scraped.drop(columns=['title', 'description', 'inclusions', 'hotel_name','agency_slug','country'], inplace=True)



In [9]:
hotels['Name'] = hotels['Name'].astype(str).apply(clean_text)

In [10]:
hotels['Name'] = hotels['Name'].astype(str).apply(clean)

In [11]:
hotels['Name'] = hotels['Name'].astype(str).apply(remove_punctuation)

In [12]:
type_labels = list(hotels.Name.unique())


In [13]:
from nltk.util import ngrams
import nltk
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords

In [14]:
tokenizer = TreebankWordTokenizer()

In [15]:
def word_grams(words, min_n=2, max_n=4):
    s = []
    for n in range(min_n, max_n):
        for ngram in ngrams(words, n):
            s.append(' '.join(str(i) for i in ngram))
    return s

In [16]:
def remove_stop_words(tokens):
    return [t for t in tokens if t not in stopwords.words('english')]

In [17]:
scraped['derived'] =scraped['corpus'].apply(remove_punctuation)

In [18]:
scraped['derived'] =scraped['derived'].apply(clean)

In [19]:
scraped['derived'] =scraped['derived'].apply(clean_text)

In [20]:
scraped['derived'] =scraped['derived'].apply(tokenizer.tokenize)

In [21]:
scraped['derived'] =scraped['derived'].apply(remove_stop_words)

In [22]:
scraped['derived'] =scraped['derived'].apply(word_grams)

In [23]:
# Verify Actions
pd.set_option('display.max_colwidth', -1)      # set to display full text in df
scraped['derived'].head(15)

C:\Users\user\AppData\Local\Temp\ipykernel_11852\170408937.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)      # set to display full text in df


0     [adaaran select, select meedhupparu, meedhupparu nan, nan maldives, maldives holiday, holiday package, package includes, includes return, return flights, flights johannesburg, johannesburg approximate, approximate taxes, taxes return, return domestic, domestic flight, flight speedboat, speedboat transfers, transfers 7, 7 nights, nights accommodation, accommodation 4star, 4star adaaran, adaaran select, select meedhupparu, meedhupparu beachfront, beachfront villa, villa meals, meals selected, selected alcoholic, alcoholic nonalcoholic, nonalcoholic drinks, drinks daily, daily exclusions, exclusions items, items personal, personal nature, nature items, items mentioned, mentioned gratuities, gratuities porterage, porterage meals, meals drinks, drinks mentioned, mentioned tourssightseeing, tourssightseeing mentioned, mentioned car, car hire, hire promise, promise incomparable, incomparable holiday, holiday tropics, tropics adaaran, adaaran select, select meedhupparu, meedhupparu one, 

In [ ]:
scraped.head()

In [24]:
def fin(text):
  res = [ele1 for ele1 in test_list1 if any(ele2 in ele1 for ele2 in text)]
  return res


In [26]:
test_list1=type_labels

In [ ]:
scraped['derived'] =scraped['derived'].apply(fin)

In [30]:
def main(text):
    result = remove_punctuation(text)
    result2=clean(result)
    test_list1=type_labels
    tokens =tokenizer.tokenize(result2)
    no_stop =remove_stop_words(tokens)
    stop_gram=word_grams(no_stop)
    tel=stop_gram
    res = [ele1 for ele1 in test_list1 if any(ele2 in ele1 for ele2 in tel)]
    return res

In [31]:
scraped['derived'] =scraped['corpus'].apply(main)


KeyboardInterrupt: 